# Permeability using OpenPNM from saved network files

### Start by importing the necessary packages

In [1]:
import numpy as np
import openpnm as op
import porespy as ps
np.set_printoptions(precision=4)
np.random.seed(10)
op.__version__

'2.4.2'

### Import network in OpenPNM

In [2]:
name = "Parker"
resolution = 2.25e-6

### Import .PNM network

In [ ]:
proj = op.io.OpenpnmIO.load_project("ValidationPNM/"+name+".pnm");
pn = proj.network;

### Import .JSON network

Now we can print the network to see how the transferred worked:

In [4]:
print(pn)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
openpnm.network.GenericNetwork : net_01
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#     Properties                                    Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1     pore.area                                     44208 / 44208
2     pore.coords                                   44208 / 44208
3     pore.diameter                                 44208 / 44208
4     pore.index                                    44208 / 44208
5     pore.volume                                   44208 / 44208
6     throat.area                                   69226 / 69226
7     throat.conns                                  69226 / 69226
8     throat.diameter                               69226 / 69226
9     throat.length                                 69226 / 69226
10    throat.perimeter                              69226 / 6922

### Check network health

Remove isolated pores or cluster of pores from the network by checking it network health. Make sure ALL keys in network health functions have no value. 

In [5]:
h = pn.check_network_health()
op.topotools.trim(network=pn, pores=h['trim_pores'])
h = pn.check_network_health()
print(h)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
key                                 value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
disconnected_clusters               []
isolated_pores                      []
trim_pores                          []
duplicate_throats                   []
bidirectional_throats               []
headless_throats                    []
looped_throats                      []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


### Assign geometry

In [6]:
geo = op.geometry.GenericGeometry(network=pn, pores=pn.Ps, throats=pn.Ts)

### Assign phase

In this example air is considered as fluid passing through porous channels. 

In [7]:
water = op.phases.Water(network=pn)

### Assign physics

In [8]:
phys_water=op.physics.GenericPhysics(network=pn, phase=water, geometry=geo)
R = geo['throat.diameter']/2.0;
L = geo['throat.length'];
phys_water['throat.hydraulic_conductance'] = (np.pi*(R**4))/(8*water['pore.viscosity'].max()*L) 

### Calculate effective permeability

In [9]:
A = (1000*1000) *resolution**2
L = 1000 * resolution
mu = water['pore.viscosity'].max()
Pressure = 10e3;
delta_P = Pressure - 0
#X
perm = op.algorithms.StokesFlow(network=pn, project=proj)
perm.setup(phase=water)
perm.set_value_BC(pores=pn.pores('top'), values=0)
perm.set_value_BC(pores=pn.pores('bottom'), values=Pressure)
perm.run()
Q = perm.rate(pores=pn.pores('bottom'), mode='group')
K = Q * L * mu / (A * delta_P)
print('X:', K/0.98e-12*1000, 'mD')

#y
perm = op.algorithms.StokesFlow(network=pn, project=proj)
perm.setup(phase=water)
perm.set_value_BC(pores=pn.pores('front'), values=0)
perm.set_value_BC(pores=pn.pores('back'), values=Pressure)
perm.run()
Q = perm.rate(pores=pn.pores('back'), mode='group')
K = Q * L * mu / (A * delta_P)
print('Y:', K/0.98e-12*1000, 'mD')

#z
perm = op.algorithms.StokesFlow(network=pn, project=proj)
perm.setup(phase=water)
perm.set_value_BC(pores=pn.pores('left'), values=0)
perm.set_value_BC(pores=pn.pores('right'), values=Pressure)
perm.run()
Q = perm.rate(pores=pn.pores('right'), mode='group')
K = Q * L * mu / (A * delta_P)
print('Z:', K/0.98e-12*1000, 'mD')

X: [10.6251] mD
Y: [12.9731] mD
Z: [14.1473] mD
